In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset
df = pd.read_csv("classified_apk_dataset.csv")

# Convert class labels to numeric values
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])

# Convert permissions to binary features
df["Permissions"] = df["Permissions"].apply(lambda x: x.split(", "))
all_permissions = set(p for sublist in df["Permissions"] for p in sublist)

for permission in all_permissions:
    df[permission] = df["Permissions"].apply(lambda perms: 1 if permission in perms else 0)

# Drop non-numeric columns
df.drop(columns=["APK_Name", "Permissions", "Activities", "Services"], inplace=True)

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(df.drop(columns=["Class"]))
y = df["Class"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Define LCNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 classes: Malware, Spyware, Scam App, Normal App
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [4]:
# Reshape input for LCNN
X_train_lcnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lcnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Train model
model.fit(X_train_lcnn, y_train, epochs=20, batch_size=32, validation_data=(X_test_lcnn, y_test))

Epoch 1/20


25/25 [==============================] - 2s 15ms/step - loss: 1.3965 - accuracy: 0.2700 - val_loss: 1.3817 - val_accuracy: 0.2850
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 1.3625 - accuracy: 0.3275 - val_loss: 1.3905 - val_accuracy: 0.3200
Epoch 3/20
25/25 [==============================] - 0s 6ms/step - loss: 1.3424 - accuracy: 0.3300 - val_loss: 1.4011 - val_accuracy: 0.2750
Epoch 4/20
25/25 [==============================] - 0s 5ms/step - loss: 1.3174 - accuracy: 0.3663 - val_loss: 1.4157 - val_accuracy: 0.3000
Epoch 5/20
25/25 [==============================] - 0s 5ms/step - loss: 1.2939 - accuracy: 0.3988 - val_loss: 1.4470 - val_accuracy: 0.2700
Epoch 6/20
25/25 [==============================] - 0s 5ms/step - loss: 1.2483 - accuracy: 0.4375 - val_loss: 1.4619 - val_accuracy: 0.2600
Epoch 7/20
25/25 [==============================] - 0s 5ms/step - loss: 1.2110 - accuracy: 0.4512 - val_loss: 1.4926 - val_accuracy: 0.2950
Epoch 8/20
25/25 

In [5]:
# Save the entire model
model.save("lcnn_apk_detection_model.h5")
print("Model saved successfully as lcnn_apk_detection_model.h5")

Model saved successfully as lcnn_apk_detection_model.h5


D:\2025\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from tensorflow.keras.models import load_model
model = load_model("lcnn_apk_detection_model.h5")
print("Model loaded successfully!")

Model loaded successfully!


In [7]:
# Predict on test data
y_pred = model.predict(X_test_lcnn)
predicted_classes = np.argmax(y_pred, axis=1)

# Convert predictions back to labels
predicted_labels = label_encoder.inverse_transform(predicted_classes)
print("Predicted Classes:", predicted_labels[:10])  # Show first 10 predictions

7/7 [==============================] - 0s 2ms/step
Predicted Classes: ['Malware' 'Malware' 'Malware' 'Spyware' 'Spyware' 'Spyware' 'Scam App'
 'Malware' 'Malware' 'Malware']


In [8]:
import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("lcnn_apk_detection_model.h5")

# Define sample data for a normal APK
normal_app_features = np.array([
    [0, 1, 0, 1, 0, 0, 1, 0, 1, 1,  # Sample binary permission encoding
     1, 2, 1, 1, 1, 1, 0, 0, 1, 0,0]  # Add more feature values if needed
])

# Reshape input for LCNN model
normal_app_features = normal_app_features.reshape(1, normal_app_features.shape[1], 1)

# Predict class
y_pred = model.predict(normal_app_features)
predicted_class = np.argmax(y_pred, axis=1)

# Convert predicted class to label
class_mapping = {0: "Normal App", 1: "Scam App", 2: "Spyware", 3: "Malware"}
predicted_label = class_mapping[predicted_class[0]]

print(f"Prediction Result: {predicted_label}")

1/1 [==============================] - 0s 92ms/step
Prediction Result: Malware


In [9]:
X_train_lcnn

array([[[ 0.87032709],
        [-1.16316   ],
        [ 0.84921752],
        ...,
        [ 0.86501546],
        [ 0.86148725],
        [ 0.88281336]],

       [[-1.1489933 ],
        [-1.16316   ],
        [ 0.84921752],
        ...,
        [-1.1560487 ],
        [-1.16078329],
        [-1.13274226]],

       [[ 0.87032709],
        [-1.16316   ],
        [-1.1775546 ],
        ...,
        [-1.1560487 ],
        [-1.16078329],
        [-1.13274226]],

       ...,

       [[ 0.87032709],
        [-1.16316   ],
        [-1.1775546 ],
        ...,
        [-1.1560487 ],
        [-1.16078329],
        [-1.13274226]],

       [[-1.1489933 ],
        [-1.16316   ],
        [-1.1775546 ],
        ...,
        [ 0.86501546],
        [-1.16078329],
        [ 0.88281336]],

       [[-1.1489933 ],
        [ 0.85972695],
        [ 0.84921752],
        ...,
        [-1.1560487 ],
        [-1.16078329],
        [ 0.88281336]]])

In [10]:
df

Class  RECORD_AUDIO  INTERNET  READ_CLIPBOARD  CAPTURE_VIDEO_OUTPUT  \
0        1             0         0               0                     0   
1        2             1         1               1                     1   
2        0             1         1               1                     0   
3        0             1         0               0                     0   
4        3             0         0               1                     0   
..     ...           ...       ...             ...                   ...   
995      1             0         1               1                     1   
996      3             1         0               1                     0   
997      3             0         1               1                     1   
998      3             1         1               1                     1   
999      2             0         0               1                     0   

     READ_PRIVILEGED_PHONE_STATE  INSTALL_PACKAGES  WRITE_SECURE_SETTINGS  \
0                              0                 1                      0   
1                              1                 0                      1   
2                              0                 0                      1   
3                              1                 0                      0   
4                              1                 1                      1   
..                           ...               ...                    ...   
995                            1                 0                      1   
996                            0                 0                      0   
997                            1                 1                      1   
998                            0                 1                      0   
999                            1                 1                      0   

     WRITE_EXTERNAL_STORAGE  MODIFY_AUDIO_SETTINGS  ...  READ_SMS  \
0                         0                      0  ...         0   
1                         1                      1  ...         1   
2                         0                      0  ...         1   
3                         0                      0  ...         0   
4                         1                      0  ...         1   
..                      ...                    ...  ...       ...   
995                       1                      1  ...         0   
996                       0                      0  ...         0   
997                       0                      0  ...         1   
998                       0                      1  ...         0   
999                       0                      1  ...         1   

     ACCESS_NETWORK_STATE  ACCESS_FINE_LOCATION  BIND_ACCESSIBILITY_SERVICE  \
0                       1                     0                           0   
1                       0                     1                           1   
2                       0                     1                           0   
3                       0                     0                           0   
4                       1                     1                           0   
..                    ...                   ...                         ...   
995                     1                     1                           1   
996                     0                     1                           0   
997                     1                     1                           1   
998                     1                     1                           1   
999                     0                     0                           0   

     READ_CONTACTS  USE_FULL_SCREEN_INTENT  CAPTURE_SECURE_VIDEO_OUTPUT  \
0                1                       0                            0   
1                0                       1                            1   
2                1                       0                            0   
3                0                       1                            0   
4             

In [11]:
df.to_csv('preprocessed.csv')